**ЛР № 9 Основы парсинга и работы с API**

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной.

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [65]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
KEYWORDS = ['gitops','2020']
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')
lst1=[]
lst2=[]
lst3=[]
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    prewie=post.find('div', class_='post__text')
    prewie_lower = prewie.text.lower()
    if any([desired in prewie_lower for desired in KEYWORDS]):
        title_element = post.find('a', class_='post__title_link')
        time_block = post.find('span', class_='post__time')
        lst1.append(time_block.text)
        lst2.append(title_element.text)
        lst3.append(title_element.attrs.get('href'))
        continue
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
        hub_lower = hub.text.lower()
        if any([hub_lower in desired for desired in KEYWORDS]):
            title_element = post.find('a', class_='post__title_link')
            time_block = post.find('span', class_='post__time')
            lst1.append(time_block.text)
            lst2.append(title_element.text)
            lst3.append(title_element.attrs.get('href'))
            continue
            
habr_posts = pd.DataFrame(list(zip(lst1, lst2, lst3)),

               columns =['data', 'title', 'link'])

habr_posts

,data,title,link
0,сегодня в 05:23,Что такое GitOps? Расширяем DevOps на Kubernet...,https://habr.com/ru/company/southbridge/blog/5...
1,сегодня в 01:15,Electron — десять спутников. Запуски года: 85 ...,https://habr.com/ru/post/525526/
2,вчера в 21:00,Синтетические данные: улучшение алгоритмов вос...,https://habr.com/ru/company/itelma/blog/525470/
3,вчера в 20:10,Как пандемия меняет ландшафт киберугроз,https://habr.com/ru/company/trendmicro/blog/52...
4,вчера в 19:38,Шесть степеней свободы: 3D object detection и ...,https://habr.com/ru/company/ods/blog/522836/
5,вчера в 18:59,Звездный уборщик: в Европе разрабатывают сборщ...,https://habr.com/ru/company/selectel/blog/525482/
